# Preprocessing Step (Etapa de Pré-Processamento)

## Table of Contents
* [Packages](#1)
* [Preprocessing Utility Function](#2)
* [Preprocessing](#3)

<a name="1"></a>
## Packages (Pacotes)

In [243]:
import os

import pandas as pd
import string
import re

<a name="2"></a>
## Preprocessing Utility Functions (Funções Utilitárias de Pré-processamento)

In [ ]:
def preprocess(dataset):
    """
    
    """
    dataset = dataset['description'].str.lower()
    

<a name="3"></a>
## Preprocessing (Pré-processamento)

In [227]:
X_comics = pd.read_csv('../data/raw/comics_corpus.csv').drop_duplicates()
X_char = pd.read_csv('../data/raw/char_corpus.csv')
X_comics.head()

,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,94801,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
2,94802,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
3,94803,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
4,94804,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...


In [229]:
X_char.head()

,id,name,description
0,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...
1,1009144,A.I.M.,AIM is a terrorist organization bent on destro...
2,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie..."
3,1010354,Adam Warlock,Adam Warlock is an artificially created human ...
4,1011031,Agent X (Nijo),Originally a partner of the mind-altering assa...


In [231]:
X_comics.shape

(18802, 3)

In [233]:
X_comics['description'].str.lower()

0        in the shadow of kirisaki mountain?a secret hi...
1        in the shadow of kirisaki mountain?a secret hi...
2        in the shadow of kirisaki mountain?a secret hi...
3        in the shadow of kirisaki mountain?a secret hi...
4        in the shadow of kirisaki mountain?a secret hi...
                               ...                        
18797    a collection of epic stories featuring human t...
18798    dr. jack castle investigates a series of murde...
18799    a historic comic series dating all the way bac...
18800    the one that started it all, featuring the fir...
18801    concluding this chilling chapter of stephen ki...
Name: description, Length: 18802, dtype: object

In [241]:
punctuations = string.punctuation
print(f'Punctuations: {punctuations}')

Punctuations: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
